# StyleGAN3 from NVIDIA

**Notes**
* To see the original code from NVIDIA [Check here](https://github.com/NVlabs/stylegan3)
* We are using a pretrained model and fine-tuning on top of it.
* If you come across bugs please post them in [Discord](https://discord.com/invite/awREd7EtMA)

---

If you find this notebook useful, consider signing up for my [Code Sprout Newsletter](https://codesprout.substack.com/welcome) or [Check my links](https://shyambv.bio.link/)

Medium article related to it is mentioned []()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


# Setup

In [2]:
 # "excellent-shard-422915-n5"
from google.colab import auth

# PROJECT_ID = "excellent-shard-422915-n5"  # @param {type:"string"}

auth.authenticate_user()

!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list

!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -

!apt -qq update

!apt -qq install gcsfuse

!mkdir colab_directory

!gcsfuse --implicit-dirs testopolito colab_directory

!ls colab_directory

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2659  100  2659    0     0  32794      0 --:--:-- --:--:-- --:--:-- 32827
OK
46 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: http://packages.cloud.google.com/apt/dists/gcsfuse-bionic/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 46 not upgraded.
Need to get 10.4 MB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../gcsfuse_2.0.1_amd64.deb ...
Unpacking gcsfuse (2.0.1) ...
Setting up gcsfuse (2.0.1) ...
{"timestamp":{"seconds":1716360253,"nanos"

In [3]:
!pip install einops ninja gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 8.4 MB/s eta 0:00:00


ONLY USE WITH GOOGLE DRIVE

In [ ]:
import os
if os.path.isdir('/content/colab_directory/WIP/stylegan3/'):
    %cd '/content/colab_directory/WIP/stylegan3/'
else:
    !git clone https://github.com/bvshyam/stylegan3.git /content/colab_directory/WIP/stylegan3/
    %cd '/content/colab_directory/WIP/stylegan3/'


Cloning into '/content/colab_directory/WIP/stylegan3'...
remote: Enumerating objects: 193, done.
error: unable to open /content/colab_directory/WIP/stylegan3/.git/objects/a3: Interrupted system call
remote: Total 193 (delta 0), reused 0 (delta 0), pack-reused 193
error: unable to open /content/colab_directory/WIP/stylegan3/.git/objects/31: Interrupted system call
Receiving objects: 100% (193/193), 4.17 MiB | 1.72 MiB/s, done.
Resolving deltas: 100% (88/88), done.
fatal: cannot create directory at 'training': Interrupted system call
You can inspect what was checked out with 'git status'
and retry with 'git restore --source=HEAD :/'

/content/colab_directory/WIP/stylegan3


If you wanted to download the current dataset execute below code

In [ ]:
!bash /content/drive/MyDrive/WIP/stargan-v2-download/download.sh afhq-dataset


bash: /content/drive/MyDrive/WIP/stargan-v2-download/download.sh: No such file or directory


# Custom Dataset

Execute the dataset_tool.py and provide the source and destination of the input dataset. Also mention the resolution required.

Examples:



```
python dataset_tool.py --source=~/downloads/afhqv2/train/cat --dest=~/datasets/afhqv2cat-512x512.zip
python dataset_tool.py --source=~/downloads/afhqv2/train/dog --dest=~/datasets/afhqv2dog-512x512.zip
python dataset_tool.py --source=~/downloads/afhqv2/train/wild --dest=~/datasets/afhqv2wild-512x512.zip
```




In [ ]:
!python /content/colab_directory/WIP/stylegan3/dataset_tool.py --source=/content/colab_directory/CRC_WSIs_original/dataset --dest=/content/colab_directory/WIP/stylegan3/datasets/wsis.zip --resolution='512x512'


100% 14269/14269 [44:45<00:00,  5.31it/s]


In [ ]:
#Unzip the transformed data if you wanted to verify
!unzip /content/drive/MyDrive/WIP/stylegan3/datasets/wsis.zip -d /content/drive/MyDrive
# Already done as normal folder, not zipped

unzip:  cannot find or open /content/drive/MyDrive/CRC_WSIs_compressed_labels/images/destination_train, /content/drive/MyDrive/CRC_WSIs_compressed_labels/images/destination_train.zip or /content/drive/MyDrive/CRC_WSIs_compressed_labels/images/destination_train.ZIP.


# Model training

You can start from a pre-trained model. Below are some of the models from Nvdia



```
stylegan3-t-ffhq-1024x1024.pkl, stylegan3-t-ffhqu-1024x1024.pkl, stylegan3-t-ffhqu-256x256.pkl
stylegan3-r-ffhq-1024x1024.pkl, stylegan3-r-ffhqu-1024x1024.pkl, stylegan3-r-ffhqu-256x256.pkl
stylegan3-t-metfaces-1024x1024.pkl, stylegan3-t-metfacesu-1024x1024.pkl
stylegan3-r-metfaces-1024x1024.pkl, stylegan3-r-metfacesu-1024x1024.pkl
stylegan3-t-afhqv2-512x512.pkl
stylegan3-r-afhqv2-512x512.pkl
```



In [ ]:
!python train.py --help

Usage: train.py [OPTIONS]

  Train a GAN using the techniques described in the paper "Alias-Free
  Generative Adversarial Networks".

  Examples:

  # Train StyleGAN3-T for AFHQv2 using 8 GPUs.
  python train.py --outdir=~/training-runs --cfg=stylegan3-t --data=~/datasets/afhqv2-512x512.zip \
      --gpus=8 --batch=32 --gamma=8.2 --mirror=1

  # Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
  python train.py --outdir=~/training-runs --cfg=stylegan3-r --data=~/datasets/metfacesu-1024x1024.zip \
      --gpus=8 --batch=32 --gamma=6.6 --mirror=1 --kimg=5000 --snap=5 \
      --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-ffhqu-1024x1024.pkl

  # Train StyleGAN2 for FFHQ at 1024x1024 resolution using 8 GPUs.
  python train.py --outdir=~/training-runs --cfg=stylegan2 --data=~/datasets/ffhq-1024x1024.zip \
      --gpus=8 --batch=32 --gamma=10 --mirror=1 --aug=noaug

Options:
  --outdir DIR   

## Using pre-trainined model

In [ ]:
# Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
!python /content/colab_directory/WIP/stylegan3/train.py --outdir=/content/colab_directory/WIP/stylegan3/training-runs --cfg=stylegan3-r --data=/content/colab_directory/WIP/stylegan3/datasets/wsis.zip \
    --gpus=1 --batch=16 --batch-gpu=8 --gamma=6.6 --mirror=False --kimg=1 --snap=1 --tick 10 --metrics=ppl2_wend --cond=True\
    # --resume=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl



Training options:
{
  "G_kwargs": {
    "class_name": "training.networks_stylegan3.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "channel_base": 65536,
    "channel_max": 1024,
    "magnitude_ema_beta": 0.9994456359721023,
    "conv_kernel": 1,
    "use_radial_filters": true
  },
  "D_kwargs": {
    "class_name": "training.networks_stylegan2.Discriminator",
    "block_kwargs": {
      "freeze_layers": 0
    },
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32768,
    "channel_max": 512
  },
  "G_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.0025
  },
  "D_opt_kwargs": {
    "class_name": "torch.optim.Adam",
    "betas": [
      0,
      0.99
    ],
    "eps": 1e-08,
    "lr": 0.002
  },
  "loss_kwargs": {
    "class_name": "training.loss.StyleGAN2Loss",
    "r1_gamma": 6.6,
    "blur_init_si

## Finetuning a model

In [ ]:
batch_size = 16
batch_gpu_size = 8 # Batch size for GPU

dataset_path = '/content/colab_directory/WIP/stylegan3/datasets/wsis.zip'
#resume_from = '/content/colab_directory/WIP/stylegan3/models/stylegan3-t-afhqv2-512x512.pkl'  #Can be the link from nvidia
resume_from ='/content/colab_directory/WIP/stylegan3/training-runs/00000-stylegan3-r-wsis-gpus1-batch16-gamma6.6/network-a657-r-512-148863.pkl'


gamma_value = 50.0 #Adjustable R1 regularization weight
snapshot_count = 8

In [ ]:

# Fine-tune StyleGAN3-R for MetFaces-U using 1 GPU, starting from the pre-trained FFHQ-U pickle.
!python /content/drive/MyDrive/WIP/stylegan3/train.py --outdir=/content/drive/MyDrive/WIP/stylegan3/results --cfg=stylegan3-t \
--data=$dataset_path --gpus=1 --batch=$batch_size --batch-gpu=$batch_gpu_size --gamma=$gamma_value --kimg=1 \
--snap=$snapshot_count --resume=$resume_from



In [ ]:
#Visualization - does not work on colab
#!python /content/drive/MyDrive/WIP/stylegan3/visualizer.py

# Generate images

In [ ]:
model_path = '/content/colab_directory/WIP/stylegan3/training-runs/00000-stylegan3-r-wsis-gpus1-batch16-gamma6.6/network-snapshot-000001.pkl'

In [ ]:
#Random generation of images using model created

!python /content/colab_directory/WIP/stylegan3/gen_images.py --outdir=/content/colab_directory/WIP/stylegan3/tmp --trunc=1 --seeds=40 \
    --network=$model_path

Loading networks from "/content/colab_directory/WIP/stylegan3/training-runs/00017-stylegan3-r-wsis-gpus1-batch16-gamma6.6/network-snapshot-000001.pkl"...
Generating image for seed 40 (0/1) ...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.


# Visualize average 2D power spectra

In [ ]:
# Calculate dataset mean and std, needed in subsequent steps.
!python /content/colab_directory/WIP/stylegan3/avg_spectra.py stats --source=/content/colab_directory/WIP/stylegan3/datasets/wsis.zip

# Calculate average spectrum for the training data.
!python /content/colab_directory/WIP/stylegan3/avg_spectra.py calc --source=/content/colab_directory/WIP/stylegan3/datasets/wsis.zip \
    --dest=/content/colab_directory/WIP/stylegan3/tmp/wsis.npz # --mean=211.314 --std=50.458

# Calculate average spectrum for a pre-trained generator.
!python /content/colab_directory/WIP/stylegan3/avg_spectra.py calc \
    --source=https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-r-afhqv2-512x512.pkl \
    --dest=/content/colab_directory/WIP/stylegan3/tmp/stylegan3-r.npz # --num=24 --mean=211.314 --std=50.458

# Display results.
!python /content/colab_directory/WIP/stylegan3/avg_spectra.py heatmap /content/colab_directory/WIP/stylegan3/tmp/wsis.npz --save /content/colab_directory/WIP/stylegan3/tmp/training-data.png
!python /content/colab_directory/WIP/stylegan3/avg_spectra.py heatmap /content/colab_directory/WIP/stylegan3/tmp/stylegan3-r.npz --save /content/colab_directory/WIP/stylegan3/tmp/stylegan3-r-hm.png
!python /content/colab_directory/WIP/stylegan3/avg_spectra.py slices /content/colab_directory/WIP/stylegan3/tmp/wsis.npz /content/colab_directory/WIP/stylegan3/tmp/stylegan3-r.npz --save /content/colab_directory/WIP/stylegan3/tmp/stylegan3-r.png

100% 24/24 [00:07<00:00,  3.06it/s]
--mean=211.314 --std=50.458
100% 24/24 [00:08<00:00,  2.96it/s]
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "filtered_lrelu_plugin"... Done.
100% 24/24 [00:02<00:00,  9.37it/s]
